In [1]:
import threading
import time

from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
import pandas as pd
import pytz
import numpy as np
import os
from sklearn import preprocessing
import re
import matplotlib
from matplotlib.patches import Polygon, Rectangle
matplotlib.use('Qt5Agg')
from datetime import timedelta
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from casadi import *
import calendar
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVR
import casadi as cd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, SGDRegressor


In [2]:
training_csv_file_path = 'D:\\mlinternship\\iitgdata\\training_data_for_svm.csv'
df = pd.read_csv(training_csv_file_path)
df

,Unnamed: 0,Time,MW,temperature,CDH,omega1,omega2,omega3,omega4,omega5,...,omega159,omega160,omega161,omega162,omega163,omega164,omega165,omega166,omega167,omega168
0,0,2023-05-03 00:00:00,3.00,22.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2023-05-03 03:00:00,2.50,27.800000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2023-05-03 05:00:00,3.00,31.500000,0.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2023-05-03 06:00:00,3.50,33.100000,2.1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2023-05-03 07:00:00,3.75,29.300000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,885,2023-06-27 19:00:00,5.70,26.900000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
886,886,2023-06-27 20:00:00,5.60,26.700000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
887,887,2023-06-27 21:00:00,5.30,26.504635,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
888,888,2023-06-27 22:00:00,4.80,26.600000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
x_unscaled = df.loc[:, 'CDH':'omega168']
x_unscaled = x_unscaled.to_numpy()
y_unscaled = df['MW']
y_unscaled = y_unscaled.to_numpy()
y_unscaled = y_unscaled.reshape(-1,1)
scaler = StandardScaler()
x = scaler.fit_transform(x_unscaled)
y = scaler.fit_transform(y_unscaled)

In [4]:
x

array([[-0.45518532, -0.03353892, -0.05815657, ..., -0.08903662,
        -0.05815657, -0.05815657],
       [-0.45518532, -0.03353892, -0.05815657, ..., -0.08903662,
        -0.05815657, -0.05815657],
       [-0.14638411, -0.03353892, -0.05815657, ..., -0.08903662,
        -0.05815657, -0.05815657],
       ...,
       [-0.45518532, -0.03353892, -0.05815657, ..., -0.08903662,
        -0.05815657, -0.05815657],
       [-0.45518532, -0.03353892, -0.05815657, ..., -0.08903662,
        -0.05815657, -0.05815657],
       [-0.45518532, -0.03353892, -0.05815657, ..., -0.08903662,
        -0.05815657, -0.05815657]])

In [5]:
y

array([[-1.41632781],
       [-1.96343835],
       [-1.41632781],
       [-0.86921726],
       [-0.59566199],
       [-0.86921726],
       [-0.86921726],
       [ 0.22500382],
       [ 0.22500382],
       [ 0.22500382],
       [ 1.04566964],
       [ 1.42864701],
       [ 1.53806912],
       [ 1.64749123],
       [ 1.42864701],
       [ 1.53806912],
       [ 0.11558171],
       [ 0.02804403],
       [-1.09900369],
       [-1.75553634],
       [-1.78836298],
       [ 0.0061596 ],
       [ 0.0061596 ],
       [-0.43152883],
       [-0.97863937],
       [ 0.55327015],
       [ 0.66269226],
       [ 1.10038069],
       [ 0.55327015],
       [ 0.49855909],
       [ 0.77211436],
       [ 0.77211436],
       [ 0.77211436],
       [ 0.49855909],
       [ 0.49855909],
       [ 0.22500382],
       [-0.04855145],
       [-0.21268461],
       [-1.06617706],
       [-0.65037305],
       [ 0.13746613],
       [ 0.01710181],
       [-0.14703135],
       [-0.13608914],
       [-0.19080019],
       [ 0

SVM Part

In [6]:
# C is inversely proportional to the number of misclassifications you allow
# epsilon is the allowed error in each instance
#Gamma parameter of RBF controls the distance of the influence of a single training point.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
y = y.reshape(len(y),)

In [8]:
stop

NameError: name 'stop' is not defined

In [ ]:
omega16df= df[df['omega16'] == 1]
x = omega16df.loc[:, 'CDH':'omega168']
X = x.to_numpy()
y = omega16df['MW']
y = y.to_numpy()
y = y.reshape(-1, 1)
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = scaler.fit_transform(y)


In [ ]:
svr = SVR(kernel='rbf', degree =3, C = 100, epsilon = 0.08, gamma =0.2)
svr.fit(X,y)
y_pred =  svr.predict(X)
mse = mean_squared_error(y,y_pred)

plt.figure(figsize=(12, 6))
plt.plot(X, y_pred, marker='o', linestyle='-', color = 'g', label = 'predicted')
plt.plot(X, y, marker='x', linestyle='-', color = 'r', label = 'actual')
plt.legend()
plt.show()

In [ ]:
'''yhat = svr_model.predict(x)
plt.ion()
plt.figure(figsize=(12, 6))
plt.plot(training_df['Time'], yhat, marker='o', linestyle='-', color = 'g')
plt.plot(training_df['Time'], y, marker='x', linestyle='-', color = 'r')
plt.xlabel('Time')
plt.ylabel('Power Consumption Changes')

plt.grid(True)
plt.legend()
plt.show()'''

In [ ]:
C = [0.01, 0.1,1,5,10,100,]
gamma = [0.0001,0.001,0.01, 0.1,1,10,100]
epsilon = [0.05, 0.1, 0.2]
degree = [2,3,4]

results = {}
for d in degree:
    for e in epsilon:
        for c in C:
            for g in gamma:
                poly_svr = SVR(kernel='poly', degree=d,epsilon=e,C=c, gamma=g)
                poly_svr.fit(x, y)
                y_pred = poly_svr.predict(X_test)
                mse = mean_squared_error(y_test, y_pred)
                score  = poly_svr.score(X_test, y_test)
                hyperparameters = {'Degree': d, 'Epsilon': e, 'C': c, 'Gamma': g}
                result_values = {'MSE': mse, 'Score': score}
                results[str(hyperparameters)] = result_values
                print("Degree: " , d, "Epsilon: ",e,"C: ", c, "gamma: ", g, "MSE: ", mse, "score: ", score)



In [ ]:
stop

In [ ]:

'''C_values = [0.01, 0.1, 1, 5, 10, 15, 50, 100, 1000]
gamma_values = [5, 100]

def fit_svr_with_timeout(C, gamma):
    start_time = time.time()

    # Create SVR with specified parameters
    poly_svr = SVR(kernel='poly', C=C, gamma=gamma)

    # Define a function to be called by a timer
    def timeout_function():
        print(f"Timeout: C={C}, gamma={gamma}")
        poly_svr.fit_timed_out = True

    timer = threading.Timer(5, timeout_function)

    try:
        timer.start()
        poly_svr.fit(x, y)
    finally:
        timer.cancel()

    elapsed_time = time.time() - start_time
    if poly_svr.fit_timed_out:
        print(f"Skipped (elapsed time: {elapsed_time:.2f} seconds): C={C}, gamma={gamma}")
        return None

    # Make predictions on the test set
    y_pred = poly_svr.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    score = poly_svr.score(X_test, y_test)

    print(f"C: {C}, gamma: {gamma}, MSE: {mse}, score: {score}, elapsed time: {elapsed_time:.2f} seconds")


# Iterate over hyperparameter combinations
for i in C_values:
    for j in gamma_values:
        fit_svr_with_timeout(i, j)
'''

In [ ]:
param_grid = {'C':[0.01, 0.1,1,5,10,15,50,100,1000],
             'kernel':['linear'],
              'epsilon':[0.05, 0.1, 0.2]}

linear_svr_grid = SVR()
# Create a GridSearchCV object
linear_grid_search = GridSearchCV(linear_svr_grid,param_grid=param_grid)

# Fit the grid search to the data
linear_grid_search.fit(x, y)

# Get the best parameters and the best model
linear_best_params = linear_grid_search.best_params_

# Make predictions on the test set
linear_y_pred = linear_grid_search.predict(X_test)
mse = mean_squared_error(y_test, linear_y_pred)

print("Best Parameters:", linear_best_params)
print("Mean Squared Error on Test Set:", mse)
print("score of the model: ", linear_grid_search.score(X_test,y_test))

In [ ]:
param_grid = {'C':[0.01, 0.1,1,5,10,15,50,100,1000],
             'kernel':['rbf'],
              'gamma': [0.0001,0.001,0.01, 0.1,1,5,10,15,50]
              }

rbf_svr_grid = SVR()
# Create a GridSearchCV object
rbf_grid_search = GridSearchCV(rbf_svr_grid,param_grid=param_grid)

# Fit the grid search to the data
rbf_grid_search.fit(x, y)

# Get the best parameters and the best model
poly_best_params = rbf_grid_search.best_params_

# Make predictions on the test set
rbf_y_pred = rbf_grid_search.predict(X_test)
mse = mean_squared_error(y_test, rbf_y_pred)

print("Best Parameters:", poly_best_params)
print("Mean Squared Error on Test Set:", mse)
print("score of the model: ", rbf_grid_search.score(X_test,y_test))

In [ ]:
stop

In [ ]:
param_grid = {'C':[0.01,0.1,1,5,10,100,1000],
             'kernel':['poly'],
              'gamma': [0.001,0.01, 0.1,1,5],
              'degree':[1,2,3,4],
              'epsilon':[0.05, 0.1, 0.2]
              }

poly_svr_grid = SVR()
# Create a GridSearchCV object
poly_grid_search = RandomizedSearchCV(poly_svr_grid,param_distributions=param_grid,n_iter = 10 ,verbose= 5)

# Fit the grid search to the data
poly_grid_search.fit(x, y)

# Get the best parameters and the best model
poly_best_params = poly_grid_search.best_params_

# Make predictions on the test set
poly_y_pred = poly_grid_search.predict(X_test)
mse = mean_squared_error(y_test, poly_y_pred)

print("Best Parameters:", poly_best_params)
print("Mean Squared Error on Test Set:", mse)
print("score of the model: ", poly_grid_search.score(X_test,y_test))

In [ ]:
stop


In [17]:
svr = SVR(kernel='rbf',  C = 100, epsilon = 0.01, gamma =10)
svr.fit(x,y)
yhat = svr.predict(x)
plt.ion()


plt.figure(figsize=(12, 6))
plt.plot(df['Time'], yhat, marker='o', linestyle='-', color = 'g', label = 'predicted')
plt.plot(df['Time'], y, marker='x', linestyle='-', color = 'r', label = 'actual')
plt.plot(df['Time'], df['CDH'], marker = '+', color = 'b', label = 'temperature')
plt.xticks(df['Time'][::12])
plt.xlabel('Time')
plt.ylabel('Power Consumption Changes')

plt.grid(True)
plt.legend()
plt.show()

accuracy = svr.score(x, y)
print("score of the model: ", accuracy)

SVR(C=100, epsilon=0.01, gamma=10)

<Figure size 1200x600 with 0 Axes>

([<matplotlib.axis.XTick at 0x20a11a57a90>,
 [Text(0.0, 0, '2023-05-03 00:00:00'),
  Text(12.0, 0, '2023-05-03 15:00:00'),
  Text(24.0, 0, '2023-05-04 09:00:00'),
  Text(36.0, 0, '2023-05-04 21:00:00'),
  Text(48.0, 0, '2023-05-05 17:00:00'),
  Text(60.0, 0, '2023-05-06 05:00:00'),
  Text(72.0, 0, '2023-05-06 17:00:00'),
  Text(84.0, 0, '2023-05-07 08:00:00'),
  Text(96.0, 0, '2023-05-07 20:00:00'),
  Text(108.0, 0, '2023-05-08 10:00:00'),
  Text(120.0, 0, '2023-05-09 06:00:00'),
  Text(132.0, 0, '2023-05-09 18:00:00'),
  Text(144.0, 0, '2023-05-10 09:00:00'),
  Text(156.0, 0, '2023-05-10 21:00:00'),
  Text(168.0, 0, '2023-05-11 14:00:00'),
  Text(180.0, 0, '2023-05-12 10:00:00'),
  Text(192.0, 0, '2023-05-13 00:00:00'),
  Text(204.0, 0, '2023-05-13 12:00:00'),
  Text(216.0, 0, '2023-05-14 06:00:00'),
  Text(228.0, 0, '2023-05-14 18:00:00'),
  Text(240.0, 0, '2023-05-15 14:00:00'),
  Text(252.0, 0, '2023-05-16 03:00:00'),
  Text(264.0, 0, '2023-05-16 15:00:00'),
  Text(276.0, 0, '2023-

Text(0.5, 0, 'Time')

Text(0, 0.5, 'Power Consumption Changes')

score of the model:  0.6462997230208756


In [19]:
svr = SVR(kernel='rbf',  C = 100, epsilon = 0.01, gamma =10)
svr.fit(x_unscaled,y_unscaled)
yhat = svr.predict(x_unscaled)
plt.ion()


plt.figure(figsize=(12, 6))
plt.plot(df['Time'], yhat, marker='o', linestyle='-', color = 'g', label = 'predicted')
plt.plot(df['Time'], y_unscaled, marker='x', linestyle='-', color = 'r', label = 'actual')
plt.plot(df['Time'], df['CDH'], marker = '+', color = 'b', label = 'temperature')
plt.xticks(df['Time'][::12])
plt.xlabel('Time')
plt.ylabel('Power Consumption Changes')

plt.grid(True)
plt.legend()
plt.show()

accuracy = svr.score(x_unscaled, y_unscaled)
print("score of the model: ", accuracy)

D:\mlinternship\power-predict\myenv\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=100, epsilon=0.01, gamma=10)

<Figure size 1200x600 with 0 Axes>

([<matplotlib.axis.XTick at 0x20a14a63b90>,
 [Text(0.0, 0, '2023-05-03 00:00:00'),
  Text(12.0, 0, '2023-05-03 15:00:00'),
  Text(24.0, 0, '2023-05-04 09:00:00'),
  Text(36.0, 0, '2023-05-04 21:00:00'),
  Text(48.0, 0, '2023-05-05 17:00:00'),
  Text(60.0, 0, '2023-05-06 05:00:00'),
  Text(72.0, 0, '2023-05-06 17:00:00'),
  Text(84.0, 0, '2023-05-07 08:00:00'),
  Text(96.0, 0, '2023-05-07 20:00:00'),
  Text(108.0, 0, '2023-05-08 10:00:00'),
  Text(120.0, 0, '2023-05-09 06:00:00'),
  Text(132.0, 0, '2023-05-09 18:00:00'),
  Text(144.0, 0, '2023-05-10 09:00:00'),
  Text(156.0, 0, '2023-05-10 21:00:00'),
  Text(168.0, 0, '2023-05-11 14:00:00'),
  Text(180.0, 0, '2023-05-12 10:00:00'),
  Text(192.0, 0, '2023-05-13 00:00:00'),
  Text(204.0, 0, '2023-05-13 12:00:00'),
  Text(216.0, 0, '2023-05-14 06:00:00'),
  Text(228.0, 0, '2023-05-14 18:00:00'),
  Text(240.0, 0, '2023-05-15 14:00:00'),
  Text(252.0, 0, '2023-05-16 03:00:00'),
  Text(264.0, 0, '2023-05-16 15:00:00'),
  Text(276.0, 0, '2023-

Text(0.5, 0, 'Time')

Text(0, 0.5, 'Power Consumption Changes')

score of the model:  0.6481945002288958


In [ ]:
svr = SVR(kernel='poly',  degree=3, C = 1000, epsilon = 0.5)
svr.fit(x,y)
yhat = svr.predict(x)
plt.ion()


plt.figure(figsize=(12, 6))
plt.plot(df['Time'], yhat, marker='o', linestyle='-', color = 'g', label = 'predicted')
plt.plot(df['Time'], y, marker='x', linestyle='-', color = 'r', label = 'actual')
plt.plot(df['Time'], df['CDH'], marker = '+', color = 'b', label = 'temperature')
plt.xticks(df['Time'][::24])
plt.xlabel('Time')
plt.ylabel('Power Consumption Changes')

plt.grid(True)
plt.legend()
plt.show()

accuracy = svr.score(x, y)
print("score of the model: ", accuracy)

In [ ]:
svr = SVR(kernel='linear',  degree=2, C = 1000, epsilon = 0.2)
svr.fit(x,y)
yhat = svr.predict(x)
plt.ion()


plt.figure(figsize=(12, 6))
plt.plot(df['Time'], yhat, marker='o', linestyle='-', color = 'g', label = 'predicted')
plt.plot(df['Time'], y, marker='x', linestyle='-', color = 'r', label = 'actual')
plt.xlabel('Time')
plt.ylabel('Power Consumption Changes')

plt.grid(True)
plt.legend()
plt.show()

accuracy = svr.score(x, y)
print("score of the model: ", accuracy)

In [ ]:

plt.figure(figsize=(12, 6))
plt.plot(df['Time'][:100], yhat[:100], marker='o', linestyle='-', color = 'g', label = 'predicted')
plt.plot(df['Time'][:100], y[:100], marker='x', linestyle='-', color = 'r', label = 'actual')
plt.xlabel('Time')
plt.ylabel('Power Consumption Changes')

plt.grid(True)
plt.legend()
plt.show()

In [11]:
svr = SVR(kernel='linear',  degree=2, C = 1000, epsilon = 0.2)
svr.fit(x,y)

omega16df= df[df['omega15'] == 1]
omega_x = omega16df.loc[:, 'CDH':'omega168']
omega_x = omega_x.to_numpy()
omega_y = omega16df['MW']
omega_y = omega_y.to_numpy()
omega_y = omega_y.reshape(-1, 1)

omega_yhat = svr.predict(omega_x)
plt.ion()

plt.figure(figsize=(12, 6))
plt.plot(omega16df['Time'], omega_yhat, marker='o', linestyle='-', color = 'g', label = 'predicted')
plt.plot(omega16df['Time'], omega_y, marker='x', linestyle='-', color = 'r', label = 'actual')
plt.xlabel('Time')
plt.ylabel('Power Consumption Changes')

plt.grid(True)
plt.legend()
plt.show()

accuracy = svr.score(x, y)
print("score of the model: ", accuracy)

SVR(C=1000, degree=2, epsilon=0.2, kernel='linear')

<Figure size 1200x600 with 0 Axes>

Text(0.5, 0, 'Time')

Text(0, 0.5, 'Power Consumption Changes')

score of the model:  0.6116461079385248


In [ ]:
stop

In [ ]:
yhat = grid_search.predict(x)
plt.ion()


plt.figure(figsize=(12, 6))
plt.plot(training_df['Time'], yhat, marker='o', linestyle='-', color = 'g', label = 'predicted')
plt.plot(training_df['Time'], y, marker='x', linestyle='-', color = 'r', label = 'actual')
plt.xlabel('Time')
plt.ylabel('Power Consumption Changes')

plt.grid(True)
plt.legend()
plt.show()